In [3]:
#conftest.py

# import pytest
import yaml
import os
from slimt import REPOSITORY, Model, Service, Package, Config

In [8]:
#Download the models:
#model_ids = ["en-es-tiny", "es-en-tiny"]
REPOSITORY.download("browsermt", "en-es-tiny")

In [9]:
REPOSITORY.download("browsermt", "es-en-tiny")

In [10]:
config_path = REPOSITORY.modelConfigPath("browsermt", "en-es-tiny")

In [11]:
config_path

'/Users/kirandevrajr/Library/Application Support/slimt/models/browsermt/enes.student.tiny11/config.slimt.yml'

In [12]:
c = None
with open(config_path) as yaml_file:
    c = yaml.safe_load(yaml_file)

In [13]:
c

{'relative-paths': True,
 'models': ['model.intgemm.alphas.bin'],
 'vocabs': ['vocab.esen.spm', 'vocab.esen.spm'],
 'shortlist': ['lex.s2t.bin', False],
 'beam-size': 1,
 'normalize': 1.0,
 'word-penalty': 0,
 'mini-batch': 64,
 'maxi-batch': 1000,
 'maxi-batch-sort': 'src',
 'workspace': 128,
 'max-length-factor': 2.0,
 'skip-cost': True,
 'gemm-precision': 'int8shiftAlphaAll',
 'ssplit-prefix-file': '',
 'ssplit-mode': 'paragraph',
 'max-length-break': 128,
 'mini-batch-words': 1024,
 'alignment': 'soft'}

In [14]:
root = os.path.dirname(config_path)

package = Package(
                model=os.path.join(root, c["models"][0]),
                vocabulary=os.path.join(root, c["vocabs"][0]),
                shortlist=os.path.join(root, c["shortlist"][0]),
            )

In [19]:
config = Config()

In [20]:
config

In [22]:
config = Config()
model = Model(config, package)

[warn] Failed to ingest expected load of Wemb_QuantMultA
[warn] Failed to ingest expected load of special:model.yml


In [23]:
sample_input_types = [
        ("1 2 3 4 5 6 7 8 9", "1 2 3 4 5 6 7 8 9", False),
        (
            "How embarrassing. A fridge full of condiments and no food.",
            "Que embarazoso. Una nevera llena de condimentos y nada de comida.",
            False,
        ),
        ("<div>Hello</div>", "<div>Hola</div>", True),
    ]

In [24]:
#Basic Translation:


translator_service = Service(workers=1, cache_size=1024)


In [50]:
from slimt.utils import toJSON

In [52]:
import ast

In [65]:
responses = translator_service.translate(model, ["whats up?", "what are you doing?"], html=False)


In [84]:

for response in responses:
    rp = response
    r = (toJSON(response, indent=4))
    r = ast.literal_eval(r)
    print(r['target']['text'])

Qué pasa?
Qué estás haciendo?


In [78]:
r

{'source': {'text': 'what are you doing?',
  'annotation': [[[0, 4], [4, 8], [8, 12], [12, 18], [18, 19], [19, 19]]]},
 'target': {'text': 'Qué estás haciendo?',
  'annotation': [[[0, 0], [0, 4], [4, 10], [10, 11], [11, 20], [20, 21]]]},
 'alignments': [[[0.9970173835754395,
    0.00037707286537624896,
    0.0017040938837453723,
    3.5690445656655356e-05,
    0.0005519440164789557,
    0.00031382308225147426],
   [0.48684170842170715,
    0.09194065630435944,
    0.09425660222768784,
    0.08864744007587433,
    0.05612228438258171,
    0.18219135701656342],
   [0.0031019493471831083,
    0.44897887110710144,
    0.16401857137680054,
    0.3526724874973297,
    0.01054419856518507,
    0.02068387344479561],
   [0.0016631616745144129,
    0.10203177481889725,
    0.5067577362060547,
    0.3745945394039154,
    9.753377526067197e-05,
    0.01485515758395195],
   [0.00046250937157310545,
    0.004688066430389881,
    0.017753154039382935,
    0.9711695313453674,
    0.0013403180055320263

In [79]:
r['target']['text']

'Qué estás haciendo?'

In [80]:
r

{'source': {'text': 'what are you doing?',
  'annotation': [[[0, 4], [4, 8], [8, 12], [12, 18], [18, 19], [19, 19]]]},
 'target': {'text': 'Qué estás haciendo?',
  'annotation': [[[0, 0], [0, 4], [4, 10], [10, 11], [11, 20], [20, 21]]]},
 'alignments': [[[0.9970173835754395,
    0.00037707286537624896,
    0.0017040938837453723,
    3.5690445656655356e-05,
    0.0005519440164789557,
    0.00031382308225147426],
   [0.48684170842170715,
    0.09194065630435944,
    0.09425660222768784,
    0.08864744007587433,
    0.05612228438258171,
    0.18219135701656342],
   [0.0031019493471831083,
    0.44897887110710144,
    0.16401857137680054,
    0.3526724874973297,
    0.01054419856518507,
    0.02068387344479561],
   [0.0016631616745144129,
    0.10203177481889725,
    0.5067577362060547,
    0.3745945394039154,
    9.753377526067197e-05,
    0.01485515758395195],
   [0.00046250937157310545,
    0.004688066430389881,
    0.017753154039382935,
    0.9711695313453674,
    0.0013403180055320263

In [81]:
def tokens(response):
    text = response["text"]
    annotations = response["annotation"][0]
    print(text, annotations)
    tokens_ = []
    for annotation in annotations:
        if annotation[0] == annotation[1]:
            continue
        tokens_.append(text[annotation[0] : annotation[1]])
    return tokens_

In [89]:
tokens(r["source"])
tokens(r["target"])

what are you doing? [[0, 4], [4, 8], [8, 12], [12, 18], [18, 19], [19, 19]]
Qué estás haciendo? [[0, 0], [0, 4], [4, 10], [10, 11], [11, 20], [20, 21]]


['Qué ', 'estás ', 'h', 'aciendo?', '']

In [88]:
# r

In [91]:
config_path_es2en = REPOSITORY.modelConfigPath("browsermt", "es-en-tiny")
c1 = None
with open(config_path_es2en) as yaml_file:
    c1= yaml.safe_load(yaml_file)

In [93]:
root = os.path.dirname(config_path_es2en)

package = Package(
                model=os.path.join(root, c1["models"][0]),
                vocabulary=os.path.join(root, c1["vocabs"][0]),
                shortlist=os.path.join(root, c1["shortlist"][0]),
            )


model = Model(config, package)

[warn] Failed to ingest expected load of Wemb_QuantMultA
[warn] Failed to ingest expected load of special:model.yml


In [94]:
responses_es = translator_service.translate(model, ["Qué estás haciendo?"], html=False)

In [96]:
for response in responses_es:
    rp = response
    r = (toJSON(response, indent=4))
    r = ast.literal_eval(r)
    print(r['target']['text'])

What are you doing?
